In [73]:
import numpy as np
import pandas as pd

In [74]:
first_table = np.array([
    [-4, -5, 0, 0, 0, 0],
    [-4, 1, 1, 0, 0, 2],
    [-2, 2, 0, 1, 0, 5],
    [3, 2, 0, 0, 1, 6],
])

variables = np.array(["x1", "x2", "s1", "s2", "s3"])

In [75]:
df = pd.DataFrame(first_table, index=["Z", "s1", "s2", "s3"], columns=["x1", "x2", "s1", "s2", "s3", "P"])

In [76]:
df

,x1,x2,s1,s2,s3,P
Z,-4,-5,0,0,0,0
s1,-4,1,1,0,0,2
s2,-2,2,0,1,0,5
s3,3,2,0,0,1,6


In [77]:
def simplex_method(table):
    basis_variables = variables[list(range(2, 5))]
    non_basis_variables = variables[list(range(0, 2))]
    z_row = table.loc["Z"]
    is_optimised_dbr = np.all(z_row[non_basis_variables] >= 0)  # умова оптимальності розв'язку
    iteration = 1

    while not is_optimised_dbr: 
        print(f"Iteration №{iteration}:")
        print()
        print("Simplex table: ")
        print(table)
        print()
        iteration += 1
        print(f"Basis variables are: {basis_variables}")
        print(f"Non-basis variables are: {non_basis_variables}")
        
        new_basis_variable = non_basis_variables[np.argmin(z_row[non_basis_variables])]
        potential_variables_out_of_basis = table.loc[:, new_basis_variable][1:][table.loc[:, new_basis_variable][1:] > 0]
        out_of_basis_variable = (table.loc[potential_variables_out_of_basis.index, "P"] / potential_variables_out_of_basis).idxmin()
        
        print(f"New basis variable is: {new_basis_variable}")
        print(f"Variable out of basis is: {out_of_basis_variable}")

        new_table = pd.DataFrame(np.zeros(table.shape), index=table.index, columns=table.columns)
        heading_elem = table.loc[out_of_basis_variable, new_basis_variable]
        
        print(f"The heading element is: {heading_elem}")
        new_table.loc[out_of_basis_variable, :] = table.loc[out_of_basis_variable, :] / heading_elem

        new_table.rename(index={out_of_basis_variable: new_basis_variable}, inplace=True)
        

        for ind in new_table.index[1:]:
            new_table.loc[ind, ind] = 1

        rows = new_table.index.drop(new_basis_variable).to_numpy()
        cols = new_table.columns.drop(rows, errors="ignore").drop(new_basis_variable).to_numpy()

        for row in rows:
            for col in cols:
                new_table.loc[row, col] = (table.loc[row, col] * heading_elem - table.loc[row, new_basis_variable] *
                                     table.loc[out_of_basis_variable, col]) / heading_elem

        basis_variables = basis_variables[basis_variables != out_of_basis_variable]
        non_basis_variables = np.append(non_basis_variables, out_of_basis_variable)
        non_basis_variables = non_basis_variables[non_basis_variables != new_basis_variable]
        basis_variables = np.append(basis_variables, new_basis_variable)

        z_row = new_table.loc["Z"]
        if (np.all(z_row[non_basis_variables] >= 0)):
            is_optimised_dbr = True
        table = new_table.copy()
        print()
    
    print('\n')
    print("The resulting table is:")
    print(table, end='\n\n')    
    print("The optimal solution is: ")
    result = pd.Series(data=np.zeros(len(variables)+1), index=np.append(variables, "Z"))
    for i in result.index:
        if i in table.index:
            result.loc[i] = table.loc[i, "P"]
    print(result)      

In [78]:
simplex_method(df)

Iteration №1:

Simplex table: 
    x1  x2  s1  s2  s3  P
Z   -4  -5   0   0   0  0
s1  -4   1   1   0   0  2
s2  -2   2   0   1   0  5
s3   3   2   0   0   1  6

Basis variables are: ['s1' 's2' 's3']
Non-basis variables are: ['x1' 'x2']
New basis variable is: x2
Variable out of basis is: s1
The heading element is: 1

Iteration №2:

Simplex table: 
      x1   x2   s1   s2   s3     P
Z  -24.0  0.0  5.0  0.0  0.0  10.0
x2  -4.0  1.0  1.0  0.0  0.0   2.0
s2   6.0  0.0 -2.0  1.0  0.0   1.0
s3  11.0  0.0 -2.0  0.0  1.0   2.0

Basis variables are: ['s2' 's3' 'x2']
Non-basis variables are: ['x1' 's1']
New basis variable is: x1
Variable out of basis is: s2
The heading element is: 6.0

Iteration №3:

Simplex table: 
     x1   x2        s1        s2   s3          P
Z   0.0  0.0 -3.000000  4.000000  0.0  14.000000
x2  0.0  1.0 -0.333333  0.666667  0.0   2.666667
x1  1.0  0.0 -0.333333  0.166667  0.0   0.166667
s3  0.0  0.0  1.666667 -1.833333  1.0   0.166667

Basis variables are: ['s3' 'x2' 'x1']
